In [1]:
import os
import pandas as pd
pd.set_option("display.max_columns", None)

In [2]:
def assemble_stats(year):
    _dir = os.listdir(year + 'eve')
    _dir = [x for x in _dir if x[0] == '2']

    # make two dfs - one building season stats and the other assembling each game senario

    season_df = pd.DataFrame(columns=['player_id', 'PAs', 'Hits'])
    senario_df = pd.DataFrame()
    game_variables = {}
    starting_pitchers = []
    pitchers = pd.DataFrame()
    
    for file in _dir:
        f = open(year + 'eve/' + file, 'r')
        print(file)
        for line in f:
            line = line.split(',')

            if line[0] == 'id':
                game_variables = {}
                starting_pitchers = []
            if line[0] == 'version':
                pass

            if line[0] == 'info':
                if line[1] == 'visteam':
                    pass
    #                 if line[2].strip() == team:
    #                     home = False
                elif line[1] == 'hometeam':
                    pass
    #                 if line[2].strip() == team:
    #                     home = True

                else:
                    if line[1] not in ['wp', 'lp', 'save']:
                        game_variables[line[1]] = line[2].strip()
            if line[0] == 'start':
                if line[3] == '0' and line[5] == '1\n':
                    starting_pitchers.append(line[1])
                    pitchers.append({'player_id': line[1]})
#                     starting_pitchers['away_SP'] = line[1]
                elif line[3] == '1' and line[5] == '1\n':
                    starting_pitchers.insert(0, line[1])
#                     starting_pitchers['home_SP'] = line[1]

            if line[0] == 'play':
                if line[6] != 'NP\n':

                    if line[3] not in season_df['player_id'].unique():
                        season_df = season_df.append({'player_id': line[3], 'PAs': 0, 'Hits': 0}, ignore_index = True)


                    if line[5][-1] == 'X':

                        if line[6][0] in ['S', 'D', 'T', 'H']:
                            season_df.loc[season_df['player_id'].isin([line[3]]), ['PAs']] += 1
                            season_df.loc[season_df['player_id'].isin([line[3]]), ['Hits']] += 1
                            senario_df = senario_df.append({**game_variables, 
                                                **{'hit': 1, 'player': line[3],
                                                 'PAs': season_df[season_df['player_id'] == line[3]]['PAs'].values[0], 
                                                   'Hits': season_df[season_df['player_id'] == line[3]]['Hits'].values[0],
                                                  'Opposing_SP': starting_pitchers[int(line[2])]}}, 
                                               ignore_index=True)
                        else:
                            season_df.loc[season_df['player_id'].isin([line[3]]), 'PAs'] += 1
                            senario_df = senario_df.append({**game_variables, 
                                                **{'hit': 0, 'player': line[3],
                                                 'PAs': season_df[season_df['player_id'] == line[3]]['PAs'].values[0], 
                                                   'Hits': season_df[season_df['player_id'] == line[3]]['Hits'].values[0],
                                                  'Opposing_SP': starting_pitchers[int(line[2])]}}, 
                                               ignore_index=True)
                            
                    elif line[5][-1] in ['C', 'S', 'B', 'T', 'H', 'M', 'V', 'K', 'U', 'Q'] and line[6][:2] not in ['WP', 'PB', 'SB', 'CS']:
                        season_df.loc[season_df['player_id'].isin([line[3]]), 'PAs'] += 1
                        senario_df = senario_df.append({**game_variables, 
                                            **{'hit': 0, 'player': line[3],
                                             'PAs': season_df[season_df['player_id'] == line[3]]['PAs'].values[0], 
                                               'Hits': season_df[season_df['player_id'] == line[3]]['Hits'].values[0],
                                                  'Opposing_SP': starting_pitchers[int(line[2])]}}, 
                                           ignore_index=True)

        
#         move later in code to allow for manipulation of cumulative stats
#         if not senario_df.empty:
#             senario_df = senario_df.sort_values('hit').drop_duplicates(subset=['date', 'number', 'player'], keep='last')
    return(senario_df, season_df)

In [ ]:
senario_df, season_df = assemble_stats('2019')
senario_df.to_csv('2019eve/senario_df.csv', index=False)
season_df.to_csv('2019eve/season_df.csv', index=False)

senario_df, season_df = assemble_stats('2020')
senario_df.to_csv('2020eve/senario_df.csv', index=False)
season_df.to_csv('2020eve/season_df.csv', index=False)

box score processing


https://widgets.sports-reference.com/wg.fcgi?css=1&site=br&url=%2Fboxes%2FLAN%2FLAN202107210.shtml&div=div_SanFranciscoGiantsbatting

https://widgets.sports-reference.com/wg.fcgi?css=1&site=br&url=%2Fboxes%2FCIN%2FCIN202107210.shtml&div=div_NewYorkMetsbatting

https://widgets.sports-reference.com/wg.fcgi?css=1&site=br&url=%2Fboxes%2FLAN%2FLAN202107210.shtml&div=div_SanFranciscoGiantspitching

https://widgets.sports-reference.com/wg.fcgi?css=1&site=br&url=%2Fboxes%2F{oppent_abr}%2F{oppent_abr}{date}.shtml&div=div_{team_full_name}{batting/pitching}

In [ ]:
# import schedules
schedule_2019 = pd.read_csv('2019SKED.TXT', columns=['date', 'doubleheader', 'day_of_week', 'visiting_team', 'visiting_league', 'v_game_number', 'home_team', 'home_league', 'h_game_number', 'daynight', 'postponed', 'makeup'])
schedule_2020 = pd.read_csv('2020REV.TXT', columns=['date', 'doubleheader', 'day_of_week', 'visiting_team', 'visiting_league', 'v_game_number', 'home_team', 'home_league', 'h_game_number', 'daynight', 'postponed', 'makeup'])
schedule_2021 = pd.read_csv('2021SKED.TXT', columns=['date', 'doubleheader', 'day_of_week', 'visiting_team', 'visiting_league', 'v_game_number', 'home_team', 'home_league', 'h_game_number', 'daynight', 'postponed', 'makeup'])


In [ ]:
# baseball-reference api

home_team = ''.upper()   # NYM
date = ''                 # 202107210
team_full_name = ''       # NewYorkMets

URL = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=br&url=%2Fboxes%2F{home_team}%2F{home_team}{date}.shtml&div=div_{team_full_name}{batting_pitching}'

In [ ]:
# Visual Crossing api (weather data)

'''Data for multiple locations can be requested in a single request
by concatenating multiple locations using the pipe (|) character.
'''
'''strat for writing api calls

group by date and time'''

URL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=24&startDateTime=2019-06-13T00:00:00&endDateTime=2019-06-20T00:00:00&unitGroup=uk&contentType=csv&dayStartTime=0:0:00&dayEndTime=0:0:00&location=Sterling,VA,US&key={api}'
